In [171]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import xarray as xr
import plotly.express as px
import statsmodels.formula.api as smf
from tqdm import tqdm
from datetime import datetime
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Input
import os
from itertools import product
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# Remit

In [172]:
# Ordnerpfad zu den CSV-Dateien
csv_folder = 'data/Json_Dateien'

# Leere Liste zum Speichern der DataFrames
dataframes = []

# Alle CSV-Dateien im Ordner einlesen
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        csv_file = os.path.join(csv_folder, filename)  # Vollständiger Pfad zur CSV-Datei
        df = pd.read_csv(csv_file)  # Einlesen der CSV-Datei
        dataframes.append(df)  # Hinzufügen des DataFrames zur Liste

# Alle DataFrames zu einem einzigen DataFrame zusammenführen
df = pd.concat(dataframes, ignore_index=True)

In [173]:
df['publishTime'] = pd.to_datetime(df['publishTime'])
df['createdTime'] = pd.to_datetime(df['createdTime'])
df['eventStatus'] = pd.to_datetime(df['eventStatus'], errors='coerce')
df['eventStartTime'] = pd.to_datetime(df['eventStartTime'], errors='coerce')
df['eventEndTime'] = pd.to_datetime(df['eventEndTime'], errors='coerce')
df['eventTime'] = df['eventEndTime'] - df['eventStartTime']

C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_14304\717224300.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['eventStatus'] = pd.to_datetime(df['eventStatus'], errors='coerce')


#### Filtern der erforderlichen Daten 

In [174]:
df_ff = df.filter(['publishTime','assetId','cause','eventTime'])
df_ff

,publishTime,assetId,cause,eventTime
0,2020-11-26 13:54:00+00:00,T_HOWAO-2,Planned Outage,0 days 09:00:00
1,2020-12-10 16:12:00+00:00,T_HOWAO-2,Planned Outage,0 days 10:00:00
2,2020-12-13 19:13:00+00:00,T_HOWAO-2,Planned Outage,0 days 02:00:00
3,2020-12-07 21:19:00+00:00,T_HOWAO-2,Planned Outage,0 days 10:00:00
4,2020-12-14 09:29:00+00:00,T_HOWAO-2,Planned Outage,0 days 03:00:00
...,...,...,...,...
431,2024-09-19 08:33:30+00:00,T_HOWAO-1,Under Investigation,1 days 23:46:00
432,2024-09-20 11:51:24+00:00,T_HOWAO-1,Under Investigation,1 days 21:46:00
433,2024-09-26 09:57:50+00:00,T_HOWAO-1,Planned Outage,6 days 04:00:00
434,2024-09-29 16:45:46+00:00,T_HOWAO-1,Under Investigation,1 days 00:00:00


##### Runden der Stunden

In [175]:
# Funktion zum Runden auf das nächste 30-Minuten-Intervall
def round_to_nearest_half_hour(dt):
    if dt.minute < 15:
        return dt.replace(minute=0, second=0)
    elif dt.minute < 45:
        return dt.replace(minute=30, second=0)
    else:
        return dt.replace(minute=0, second=0) + pd.Timedelta(hours=1)

# Funktion zum Runden auf die nächste volle Stunde
def round_to_nearest_hour(dt):
    if dt.minute < 30:
        return dt.replace(minute=0, second=0)
    else:
        return dt.replace(minute=0, second=0) + pd.Timedelta(hours=1)

# Anwendung der Rundungsfunktionen auf df_tt
df_ff['rounded_half_hour'] = df_ff['publishTime'].apply(round_to_nearest_half_hour)

df_ff = df_ff.filter(['assetId','cause'	,'eventTime',	'rounded_half_hour'	])

##### Dummies erstellen

In [176]:
df_ff = pd.get_dummies(df_ff, columns=['assetId','cause'], prefix='Category')
df_ff.head()

,eventTime,rounded_half_hour,Category_T_HOWAO-1,Category_T_HOWAO-2,Category_T_HOWAO-3,Category_Forced Outage,Category_NonOperator Alert,Category_Planned Outage,Category_Turbine,Category_Under Investigation
0,0 days 09:00:00,2020-11-26 14:00:00+00:00,False,True,False,False,False,True,False,False
1,0 days 10:00:00,2020-12-10 16:00:00+00:00,False,True,False,False,False,True,False,False
2,0 days 02:00:00,2020-12-13 19:00:00+00:00,False,True,False,False,False,True,False,False
3,0 days 10:00:00,2020-12-07 21:30:00+00:00,False,True,False,False,False,True,False,False
4,0 days 03:00:00,2020-12-14 09:30:00+00:00,False,True,False,False,False,True,False,False


#### Einlesen und vorerarbeitung der Daten

In [190]:
def pre_csv(df):
    df.dtm = pd.to_datetime(df.dtm)  
    return df

def pre_dwd(df):
    df = df.to_dataframe().reset_index().rename(columns={"ref_datetime": "reference_time", "valid_datetime": "valid_time"})
    df.reference_time = df.reference_time.dt.tz_localize("UTC")  
    df.valid_time = df.reference_time + df.valid_time * pd.Timedelta(1, "h")  
    return df

def pre_ncep(df):
    return df


df_pes_0 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20200920_20231027.nc"))
df_pes_1 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20231027_20240108.nc"))
df_pes_2 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20240108_20240129.nc"))
df_pes_3 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20240129_20240519.nc"))

df_pes = pd.concat([df_pes_0, df_pes_1, df_pes_2, df_pes_3]).sort_values(["reference_time", "valid_time"]).reset_index(drop=True)
del df_pes_0, df_pes_1, df_pes_2, df_pes_3

df_pes = df_pes.groupby(["reference_time", "valid_time"]).mean().reset_index().drop(columns=["point", "longitude", "latitude"])


df_pes = df_pes.set_index("valid_time").groupby(["reference_time"]).resample("30min").interpolate("linear").drop(columns="reference_time").reset_index()

df_hornsea_0 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20200920_20231027.nc"))
df_hornsea_1 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20231027_20240108.nc"))
df_hornsea_2 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240108_20240129.nc"))
df_hornsea_3 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240129_20240519.nc"))

df_hornsea = pd.concat([df_hornsea_0, df_hornsea_1, df_hornsea_2, df_hornsea_3]).sort_values(["reference_time", "valid_time"]).reset_index(drop=True)
del df_hornsea_0, df_hornsea_1, df_hornsea_2, df_hornsea_3

df_hornsea = df_hornsea.groupby(["reference_time", "valid_time"]).mean().reset_index().drop(columns=["longitude", "latitude"])

df_hornsea = df_hornsea.set_index("valid_time").groupby(["reference_time"]).resample("30min").interpolate("linear").drop(columns="reference_time").reset_index()

df_0 = pre_csv(pd.read_csv("data/Energy_Data_20200920_20240118.csv"))
df_1 = pre_csv(pd.read_csv("data/Energy_Data_20240119_20240519.csv"))

df = pd.concat([df_0, df_1]).sort_values(["dtm"]).reset_index(drop=True)
del df_0, df_1

df["Wind_MWh_credit"] = 0.5 * df["Wind_MW"] - df["boa_MWh"]
df["Solar_MWh_credit"] = 0.5 * df["Solar_MW"]

df_full = pd.merge(df_pes, df_hornsea, on=["reference_time", "valid_time"])

df_full = df_full.merge(df[["dtm", "Wind_MWh_credit", "Solar_MWh_credit"]], left_on="valid_time", right_on="dtm", how="left")

df_full["forcast_hours"] = (df_full.valid_time - df_full.reference_time) / pd.Timedelta(1, "h")
df_full["year"] = df_full.valid_time.dt.year
df_full["month"] = df_full.valid_time.dt.month
df_full["day"] = df_full.valid_time.dt.day
df_full["hour"] = df_full.valid_time.dt.hour

df_full["total_generation_MWh"] = df_full["Wind_MWh_credit"] + df_full["Solar_MWh_credit"]

df_train = df_full.loc[df_full.reference_time < "2023-09-01"]
df_test = df_full.loc[df_full.reference_time >= "2023-09-01"]


In [191]:
df_train = pd.merge(df_train, df_ff, left_on="valid_time", right_on= 'rounded_half_hour', how ='left')
df_test = pd.merge(df_test, df_ff, left_on="valid_time", right_on= 'rounded_half_hour', how ='left')

In [192]:
df_train.columns

Index(['reference_time', 'valid_time', 'CloudCover', 'SolarDownwardRadiation',
       'Temperature_x', 'RelativeHumidity', 'Temperature_y', 'WindDirection',
       'WindDirection:100', 'WindSpeed', 'WindSpeed:100', 'dtm',
       'Wind_MWh_credit', 'Solar_MWh_credit', 'forcast_hours', 'year', 'month',
       'day', 'hour', 'total_generation_MWh', 'eventTime', 'rounded_half_hour',
       'Category_T_HOWAO-1', 'Category_T_HOWAO-2', 'Category_T_HOWAO-3',
       'Category_Forced Outage', 'Category_NonOperator Alert',
       'Category_Planned Outage', 'Category_Turbine',
       'Category_Under Investigation'],
      dtype='object')

In [202]:
df_train = pd.read_parquet("df_train.parquet")
df_test = pd.read_parquet("df_test.parquet")

windspeed_train_pca = df_train[["WindSpeed", "WindSpeed:100"]].to_numpy()
windspeed_test_pca = df_test[["WindSpeed", "WindSpeed:100"]].to_numpy()

scale_pca_pipe = Pipeline([
    ("scaling", StandardScaler()),
    ("pca", PCA(n_components=1))
])

windspeed_train_pca = scale_pca_pipe.fit_transform(windspeed_train_pca)
windspeed_test_pca = scale_pca_pipe.transform(windspeed_test_pca)

df_train["WindSpeedPCA"] = windspeed_train_pca
df_test["WindSpeedPCA"] = windspeed_test_pca

winddirection_train_pca = df_train[["WindDirection", "WindDirection:100"]].to_numpy()
winddirection_test_pca = df_test[["WindDirection", "WindDirection:100"]].to_numpy()

winddirection_train_pca = scale_pca_pipe.fit_transform(winddirection_train_pca)
winddirection_test_pca = scale_pca_pipe.transform(winddirection_test_pca)

df_train["WindDirectionPCA"] = winddirection_train_pca
df_test["WindDirectionPCA"] = winddirection_test_pca
df_train["total_generation_MWh"] = df_train['Wind_MWh_credit'] + df_train['Solar_MWh_credit']
df_test["total_generation_MWh"] = df_test['Wind_MWh_credit'] + df_test['Solar_MWh_credit']

In [181]:
df_test.columns

Index(['reference_time', 'valid_time', 'CloudCover', 'SolarDownwardRadiation',
       'Temperature_x', 'RelativeHumidity', 'Temperature_y', 'WindDirection',
       'WindDirection:100', 'WindSpeed', 'WindSpeed:100', 'dtm',
       'Wind_MWh_credit', 'Solar_MWh_credit', 'forcast_hours', 'year', 'month',
       'day', 'hour', 'total_generation_MWh', 'eventTime', 'rounded_half_hour',
       'Category_T_HOWAO-1', 'Category_T_HOWAO-2', 'Category_T_HOWAO-3',
       'Category_Forced Outage', 'Category_NonOperator Alert',
       'Category_Planned Outage', 'Category_Turbine',
       'Category_Under Investigation', 'WindSpeedPCA', 'WindDirectionPCA'],
      dtype='object')

In [183]:
df_train.columns

Index(['reference_time', 'valid_time', 'CloudCover', 'SolarDownwardRadiation',
       'Temperature_x', 'RelativeHumidity', 'Temperature_y', 'WindDirection',
       'WindDirection:100', 'WindSpeed', 'WindSpeed:100', 'dtm',
       'Wind_MWh_credit', 'Solar_MWh_credit', 'forcast_hours', 'year', 'month',
       'day', 'hour', 'total_generation_MWh', 'eventTime', 'rounded_half_hour',
       'Category_T_HOWAO-1', 'Category_T_HOWAO-2', 'Category_T_HOWAO-3',
       'Category_Forced Outage', 'Category_NonOperator Alert',
       'Category_Planned Outage', 'Category_Turbine',
       'Category_Under Investigation', 'WindSpeedPCA', 'WindDirectionPCA'],
      dtype='object')

In [184]:
df_train.columns

Index(['reference_time', 'valid_time', 'CloudCover', 'SolarDownwardRadiation',
       'Temperature_x', 'RelativeHumidity', 'Temperature_y', 'WindDirection',
       'WindDirection:100', 'WindSpeed', 'WindSpeed:100', 'dtm',
       'Wind_MWh_credit', 'Solar_MWh_credit', 'forcast_hours', 'year', 'month',
       'day', 'hour', 'total_generation_MWh', 'eventTime', 'rounded_half_hour',
       'Category_T_HOWAO-1', 'Category_T_HOWAO-2', 'Category_T_HOWAO-3',
       'Category_Forced Outage', 'Category_NonOperator Alert',
       'Category_Planned Outage', 'Category_Turbine',
       'Category_Under Investigation', 'WindSpeedPCA', 'WindDirectionPCA'],
      dtype='object')

#### Umbennung der Spalten

In [197]:
def rename_columns(df_train, df_test, rename_dict):
    df_train.rename(columns=rename_dict, inplace=True)
    df_test.rename(columns=rename_dict, inplace=True)

# Wörterbuch für die umzubenennenden Spalten
rename_dict = {
    'Category_T_HOWAO-1': 'Category_T_HOWAO_1',
    'Category_T_HOWAO-2': 'Category_T_HOWAO_2',
    'Category_T_HOWAO-3': 'Category_T_HOWAO_3',  
    'Category_Forced Outage' : 'Category_Forced_Outage',
    'Category_NonOperator Alert': 'Category_NonOperator_Alert', 
    'Category_Planned Outage':'Category_Planned_Outage', 
    'Category_Under Investigation' : 'Category_Under_Investigation',
}

# Aufrufen der Funktion
rename_columns(df_train, df_test, rename_dict)


In [198]:
df_train.columns

Index(['reference_time', 'valid_time', 'CloudCover', 'SolarDownwardRadiation',
       'Temperature_x', 'RelativeHumidity', 'Temperature_y', 'WindDirection',
       'WindDirection:100', 'WindSpeed', 'WindSpeed:100', 'dtm',
       'Wind_MWh_credit', 'Solar_MWh_credit', 'forcast_hours', 'year', 'month',
       'day', 'hour', 'total_generation_MWh', 'eventTime', 'rounded_half_hour',
       'Category_T_HOWAO_1', 'Category_T_HOWAO_2', 'Category_T_HOWAO_3',
       'Category_Forced_Outage', 'Category_NonOperator_Alert',
       'Category_Planned_Outage', 'Category_Turbine',
       'Category_Under_Investigation', 'WindSpeedPCA'],
      dtype='object')

#### Null Werte mit 0 oder 1 Füllen

In [199]:
def process_categories(df_train, df_test, columns):
    for column in columns:
        df_train[column] = df_train[column].fillna(False)
        df_test[column] = df_test[column].fillna(False)
        df_train[column] = df_train[column].replace({True: 1, False: 0})
        df_test[column] = df_test[column].replace({True: 1, False: 0})

# Liste der zu verarbeitenden Spalten
columns_to_process = ['Category_T_HOWAO_1', 'Category_T_HOWAO_2', 'Category_T_HOWAO_3','Category_Forced_Outage',
       'Category_NonOperator_Alert', 'Category_Planned_Outage', 'Category_Turbine', 'Category_Under_Investigation',]  

# Aufrufen der Funktion
process_categories(df_train, df_test, columns_to_process)

### Modell training Wind

In [200]:
def pinball(y, q, alpha):
    return (y - q) * (alpha - (y < q))

def pinball_score(df, model_idx):
    score = []
    for qu in range(10, 100, 10):
        y_true = df['Wind_MWh_credit']
        y_pred = df[f'Model_{model_idx}_q{qu}']
        score.append(pinball(y_true, y_pred, qu / 100).mean())
    return score

data_sizes = [300000]
regularizers_faktor = [0.4, 0.2, 0.1, ]
dropout_rates_1 = [0.9,0.7,0.3, 0.1]
dropout_rates_2 = [0.1, 0.3,0.7,0.9]
epochs_options = [20,50 ]
batch_sizes = [1024]  
sequence_lengths = [12]
lstm_units_1 = [64]  
lstm_units_2 = [32] 

results_file_name = f"data/LSTM_wind.csv"


os.makedirs(os.path.dirname(results_file_name), exist_ok=True)

with open(results_file_name, 'w') as f:
    f.write('Epochs,Batch_Size,Sequence_Length,Dropout_Rate_1,Dropout_Rate_2,Regularizers_Faktor,LSTM_Units_1,LSTM_Units_2,Data_Size,Pinball_Score_Train,Pinball_Score_Test\n')

for data_size in data_sizes:
    df_train_subset = df_train.iloc[1:300001]
    df_test_subset = df_test.head(data_size)


    features = ['RelativeHumidity', 
       'WindDirection', 'WindDirection:100', 'WindSpeed', 'WindSpeed:100',
        'CloudCover', 'SolarDownwardRadiation', 'month', 'day', 'hour',
       'WindSpeedPCA', 'WindDirectionPCA',
       'Category_T_HOWAO_1', 'Category_T_HOWAO_2', 'Category_T_HOWAO_3',
       'Category_Forced_Outage', 'Category_NonOperator_Alert',
       'Category_Planned_Outage', 'Category_Turbine',
       'Category_Under_Investigation' 
       ]
    target = 'Wind_MWh_credit'

    scaler = MinMaxScaler()
    scaled_train_data = scaler.fit_transform(df_train_subset[features + [target]])

    # Sequenzen für Trainingsdaten erstellen
    X_train, y_train = [], []
    for i in range(sequence_lengths[0], len(scaled_train_data)):
        X_train.append(scaled_train_data[i-sequence_lengths[0]:i, :-1])
        y_train.append(scaled_train_data[i, -1])
    X_train, y_train = np.array(X_train), np.array(y_train)

    scaled_test_data = scaler.transform(df_test_subset[features + [target]])


    X_test, y_test = [], []
    for i in range(sequence_lengths[0], len(scaled_test_data)):
        X_test.append(scaled_test_data[i-sequence_lengths[0]:i, :-1])
        y_test.append(scaled_test_data[i, -1])
    X_test, y_test = np.array(X_test), np.array(y_test)


    
    param_combinations = list(product(
        epochs_options,
        batch_sizes,
        dropout_rates_1,
        dropout_rates_2,
        regularizers_faktor,
        lstm_units_1,
        lstm_units_2  
    ))

    for epochs, batch_size, dropout_rate_1, dropout_rate_2, reg_factor, lstm_units_1, lstm_units_2 in param_combinations:
        print(f"Trainiere Modell mit {lstm_units_1} {lstm_units_2} LSTM-Einheiten, Epochs: {epochs}, Batch Size: {batch_size}, "
            f"regularizers_faktor: {reg_factor}, Dropout Rate 1: {dropout_rate_1}, Dropout Rate 2: {dropout_rate_2}")

        model = Sequential()
        # Hier fügen wir den Input-Layer hinzu
        model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
        
        model.add(LSTM(lstm_units_1, return_sequences=True,
                    kernel_regularizer=tf.keras.regularizers.l2(reg_factor)))
        model.add(BatchNormalization())
        model.add(Dropout(dropout_rate_1))
        model.add(LSTM(lstm_units_2, return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(reg_factor)))
        model.add(BatchNormalization())
        model.add(Dropout(dropout_rate_2))
        model.add(Dense(1))

        if tf.config.list_physical_devices('GPU'):
            print("GPU gefunden. Das Modell wird auf der GPU trainiert.")
        else:
            print("Keine GPU gefunden. Das Modell wird auf der CPU trainiert.")

        model.compile(optimizer='adam', loss='mean_squared_error')
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2,
                  callbacks=[early_stopping], verbose=0)

        df_train_subset.loc[:, 'Model_1_q50'] = np.nan

        # Vorhersagen für das Training
        train_predictions = model.predict(X_train)
        df_train_subset.loc[df_train_subset.index[-len(train_predictions):], 'Model_1_q50'] = train_predictions.flatten()

        # Berechne quantilbasierte Modelle
        for qu in range(10, 100, 10):
            df_train_subset.loc[:, f"Model_1_q{qu}"] = df_train_subset['Model_1_q50'] * (qu / 100)

        # Berechne den Pinball-Score für das Training
        train_score = pinball_score(df_train_subset, 1)
        overall_pinball_score_train = sum(train_score) / len(train_score)

        # Setze Model_1_q50 auf NaN für das Test-Subset
        df_test_subset.loc[:, 'Model_1_q50'] = np.nan

        # Vorhersagen für das Testen
        test_predictions = model.predict(X_test)
        df_test_subset.loc[df_test_subset.index[-len(test_predictions):], 'Model_1_q50'] = test_predictions.flatten()

        # Berechne quantilbasierte Modelle für das Test-Subset
        for qu in range(10, 100, 10):
            df_test_subset.loc[:, f"Model_1_q{qu}"] = df_test_subset['Model_1_q50'] * (qu / 100)

        # Berechne den Pinball-Score für das Testen
        test_score = pinball_score(df_test_subset, 1)
        overall_pinball_score_test = sum(test_score) / len(test_score)

        # Ergebnisse in die CSV schreiben, einschließlich der LSTM-Einheiten
        with open(results_file_name, 'a') as f:
            f.write(f"{epochs},{batch_size},{sequence_lengths[0]},{dropout_rate_1},{dropout_rate_2},{reg_factor},{lstm_units_1},{lstm_units_2},{data_size},{overall_pinball_score_train},{overall_pinball_score_test}\n")

        results_df = pd.read_csv(results_file_name)
        best_result = results_df.loc[results_df['Pinball_Score_Test'].idxmin()]
        print(f"Beste Kombination: Epochs: {best_result['Epochs']}, Batch Size: {best_result['Batch_Size']}, "
              f"regularizers_faktor: {best_result['Regularizers_Faktor']}, LSTM Units 1: {best_result['LSTM_Units_1']},LSTM Units 2: {best_result['LSTM_Units_2']}, "
              f"Dropout Rate 1: {best_result['Dropout_Rate_1']}, Dropout Rate 2: {best_result['Dropout_Rate_2']}, "
              f"Pinball Score Test: {best_result['Pinball_Score_Test']}")


KeyError: "['WindDirectionPCA'] not in index"

In [162]:
# Definiere die Funktion für das LSTM-Modell
def create_model(=50, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))  # Ausgabe ist ein einzelner Wert
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Erstelle den KerasRegressor
model = KerasRegressor(build_fn=create_model, verbose=0)

# Definiere die Parameter für die Grid Search
param_grid = {
    'units': [50, 100],
    'dropout_rate': [0.1, 0.2],
    'batch_size': [16, 32],
    'epochs': [50, 100]
}

# Führe die Grid Search durch
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Ergebnisse anzeigen
print(f"Beste Parameter: {grid_result.best_params_}")
print(f"Bestes Ergebnis: {grid_result.best_score_}")


ModuleNotFoundError: No module named 'keras.wrappers'

In [32]:
correlation_matrix = df_train.corr()
target_correlation = correlation_matrix['total_generation_MWh'].sort_values(ascending=False)

print("Korrelation mit dem Zielwert:")
print(target_correlation)


Korrelation mit dem Zielwert:
total_generation_MWh           1.000000
Wind_MWh_credit                0.697678
Wind_MW                        0.693906
Solar_MWh_credit               0.608405
Solar_MW                       0.608405
SolarDownwardRadiation         0.502808
adjusted_solar_radiation       0.502731
WindSpeed:100                  0.493034
WindSpeedPCA                   0.487504
humidity_wind_interaction      0.475112
WindSpeed                      0.474981
temp_y_solar_interaction       0.449042
temp_x_solar_interaction       0.440999
wind_gradient                  0.439809
temp_solar                     0.187486
WindDirectionPCA               0.089220
WindDirection                  0.088963
WindDirection:100              0.088901
CloudCover                     0.044710
CloudCover_lag_1h              0.043996
hour                           0.007100
month                          0.006474
cloud_cover_change             0.004426
valid_time                     0.002092
dtm       

## Modelltraining Solar 

In [209]:
def pinball(y, q, alpha):
    return (y - q) * (alpha - (y < q))

def pinball_score(df, model_idx):
    score = []
    for qu in range(10, 100, 10):
        y_true = df['Wind_MWh_credit']
        y_pred = df[f'Model_{model_idx}_q{qu}']
        score.append(pinball(y_true, y_pred, qu / 100).mean())
    return score

# Beispielwerte
data_sizes = [300000] 
regularizers_faktor = [0.4, 0.2, 0.1]
dropout_rates_1 = [0.9, 0.7, 0.3, 0.1]
dropout_rates_2 = [0.1, 0.3, 0.7, 0.9]
epochs_options = [200, 150, 100, 50]
batch_sizes = [1024]
sequence_lengths = [12]
lstm_units_1 = [64]
lstm_units_2 = [64]

results_file_name = f"data/results_solar.csv"
os.makedirs(os.path.dirname(results_file_name), exist_ok=True)

with open(results_file_name, 'w') as f:
    f.write('Epochs,Batch_Size,Sequence_Length,Dropout_Rate_1,Dropout_Rate_2,Regularizers_Faktor,LSTM_Units_1,LSTM_Units_2,Data_Size,Pinball_Score_Train,Pinball_Score_Test\n')

for data_size in data_sizes:
    df_train_subset = df_train.iloc[:data_size]
    df_test_subset = df_test.head(data_size)

    # Feature-Liste definieren
    features = [
        'RelativeHumidity', 'CloudCover', 'SolarDownwardRadiation', 'month', 'day', 'hour', 'CloudCover',   'SolarDownwardRadiation',
      
    ]
    target = 'Solar_MWh_credit'

    # Überprüfe auf fehlende Spalten
    missing_columns = [col for col in (features + [target]) if col not in df_train_subset.columns]
    if missing_columns:
        print("Fehlende Spalten:", missing_columns)
        continue  # Überspringe diese Iteration, falls Spalten fehlen

    scaler = MinMaxScaler()
    scaled_train_data = scaler.fit_transform(df_train_subset[features + [target]])

    # Trainingssequenzen erstellen
    X_train, y_train = [], []
    for i in range(sequence_lengths[0], len(scaled_train_data)):
        X_train.append(scaled_train_data[i-sequence_lengths[0]:i, :-1])
        y_train.append(scaled_train_data[i, -1])
    X_train, y_train = np.array(X_train), np.array(y_train)

    scaled_test_data = scaler.transform(df_test_subset[features + [target]])

    # Testsequenzen erstellen
    X_test, y_test = [], []
    for i in range(sequence_lengths[0], len(scaled_test_data)):
        X_test.append(scaled_test_data[i-sequence_lengths[0]:i, :-1])
        y_test.append(scaled_test_data[i, -1])
    X_test, y_test = np.array(X_test), np.array(y_test)

    param_combinations = list(product(
        epochs_options, batch_sizes, dropout_rates_1, dropout_rates_2,
        regularizers_faktor, lstm_units_1, lstm_units_2
    ))

    for epochs, batch_size, dropout_rate_1, dropout_rate_2, reg_factor, lstm_units_1, lstm_units_2 in param_combinations:
        print(f"Trainiere Modell mit Epochs: {epochs}, Batch Size: {batch_size}, "
              f"regularizers_faktor: {reg_factor}, Dropout Rate 1: {dropout_rate_1}, Dropout Rate 2: {dropout_rate_2}")

        model = Sequential([
            Input(shape=(X_train.shape[1], X_train.shape[2])),
            LSTM(lstm_units_1, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(reg_factor)),
            BatchNormalization(),
            Dropout(dropout_rate_1),
            LSTM(lstm_units_2, return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(reg_factor)),
            BatchNormalization(),
            Dropout(dropout_rate_2),
            Dense(1)
        ])

        model.compile(optimizer='adam', loss='mean_squared_error')
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2,
                  callbacks=[early_stopping], verbose=0)

        # Vorhersagen für Trainings- und Testdaten durchführen und Pinball-Score berechnen
        df_train_subset['Model_1_q50'] = np.nan
        df_train_subset.loc[df_train_subset.index[-len(y_train):], 'Model_1_q50'] = model.predict(X_train).flatten()
        
        for qu in range(10, 100, 10):
            df_train_subset[f"Model_1_q{qu}"] = df_train_subset['Model_1_q50'] * (qu / 100)
        
        train_score = pinball_score(df_train_subset, 1)
        overall_pinball_score_train = np.mean(train_score)

        df_test_subset['Model_1_q50'] = np.nan
        df_test_subset.loc[df_test_subset.index[-len(y_test):], 'Model_1_q50'] = model.predict(X_test).flatten()

        for qu in range(10, 100, 10):
            df_test_subset[f"Model_1_q{qu}"] = df_test_subset['Model_1_q50'] * (qu / 100)

        test_score = pinball_score(df_test_subset, 1)
        overall_pinball_score_test = np.mean(test_score)

        # Ergebnisse in die CSV schreiben
        with open(results_file_name, 'a') as f:
            f.write(f"{epochs},{batch_size},{sequence_lengths[0]},{dropout_rate_1},{dropout_rate_2},{reg_factor},{units_1},{units_2},{data_size},{overall_pinball_score_train},{overall_pinball_score_test}\n")

        # Ausgabe der besten Kombination
        results_df = pd.read_csv(results_file_name)
        best_result = results_df.loc[results_df['Pinball_Score_Test'].idxmin()]
        print(f"Beste Kombination: Epochs: {best_result['Epochs']}, Batch Size: {best_result['Batch_Size']}, "
              f"regularizers_faktor: {best_result['Regularizers_Faktor']}, LSTM Units 1: {best_result['LSTM_Units_1']},LSTM Units 2: {best_result['LSTM_Units_2']}, "
              f"Dropout Rate 1: {best_result['Dropout_Rate_1']}, Dropout Rate 2: {best_result['Dropout_Rate_2']}, "
              f"Pinball Score Test: {best_result['Pinball_Score_Test']}")


Trainiere Modell mit Epochs: 100, Batch Size: 1024, regularizers_faktor: 0.4, Dropout Rate 1: 0.9, Dropout Rate 2: 0.1


C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_14304\512680382.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_subset['Model_1_q50'] = np.nan


9375/9375 ━━━━━━━━━━━━━━━━━━━━ 63s 7ms/step
   1/7852 ━━━━━━━━━━━━━━━━━━━━ 3:31 27ms/step

C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_14304\512680382.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_subset[f"Model_1_q{qu}"] = df_train_subset['Model_1_q50'] * (qu / 100)
C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_14304\512680382.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_subset[f"Model_1_q{qu}"] = df_train_subset['Model_1_q50'] * (qu / 100)
C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_14304\512680382.py:106: SettingWithCopyWarning: 
A va

7852/7852 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step


...

C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_14304\512680382.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_subset['Model_1_q50'] = np.nan


9375/9375 ━━━━━━━━━━━━━━━━━━━━ 101s 11ms/step


C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_14304\512680382.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_subset[f"Model_1_q{qu}"] = df_train_subset['Model_1_q50'] * (qu / 100)
C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_14304\512680382.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_subset['Model_1_q50'] = np.nan


2341/7852 ━━━━━━━━━━━━━━━━━━━━ 1:04 12ms/step

KeyboardInterrupt: 

## Modelltraining gesamt

In [ ]:
def pinball(y, q, alpha):
    return (y - q) * (alpha - (y < q))

def pinball_score(df, model_idx):
    score = []
    for qu in range(10, 100, 10):
        y_true = df['Wind_MWh_credit']
        y_pred = df[f'Model_{model_idx}_q{qu}']
        score.append(pinball(y_true, y_pred, qu / 100).mean())
    return score


data_sizes = [300000] 
regularizers_faktor = [0.4, 0.2, 0.1]
dropout_rates_1 = [0.9, 0.7, 0.3, 0.1]
dropout_rates_2 = [0.1, 0.3, 0.7, 0.9]
epochs_options = [200, 150, 100, 50]
batch_sizes = [1024]
sequence_lengths = [12]
lstm_units_1 = [64]
lstm_units_2 = [64]

results_file_name = f"data/results_gesamt.csv"
os.makedirs(os.path.dirname(results_file_name), exist_ok=True)

with open(results_file_name, 'w') as f:
    f.write('Epochs,Batch_Size,Sequence_Length,Dropout_Rate_1,Dropout_Rate_2,Regularizers_Faktor,LSTM_Units_1,LSTM_Units_2,Data_Size,Pinball_Score_Train,Pinball_Score_Test\n')

for data_size in data_sizes:
    df_train_subset = df_train.iloc[:data_size]
    df_test_subset = df_test.head(data_size)

    # Feature-Liste definieren
    features = [
        'RelativeHumidity', 'CloudCover', 'SolarDownwardRadiation', 'month', 'day', 'hour', 'CloudCover', 'SolarDownwardRadiation',
        'WindDirection', 'WindDirection:100', 'WindSpeed', 'WindSpeed:100',
        'CloudCover', 'SolarDownwardRadiation', 'month', 'day', 'hour',
       'WindSpeedPCA', 'WindDirectionPCA',
       'Category_T_HOWAO_1', 'Category_T_HOWAO_2', 'Category_T_HOWAO_3',
       'Category_Forced_Outage', 'Category_NonOperator_Alert',
       'Category_Planned_Outage', 'Category_Turbine',
       'Category_Under_Investigation'
      
    ]
    target = 'total_generation_MWh'

    # Überprüfe auf fehlende Spalten
    missing_columns = [col for col in (features + [target]) if col not in df_train_subset.columns]
    if missing_columns:
        print("Fehlende Spalten:", missing_columns)
        continue  # Überspringe diese Iteration, falls Spalten fehlen

    scaler = MinMaxScaler()
    scaled_train_data = scaler.fit_transform(df_train_subset[features + [target]])

    # Trainingssequenzen erstellen
    X_train, y_train = [], []
    for i in range(sequence_lengths[0], len(scaled_train_data)):
        X_train.append(scaled_train_data[i-sequence_lengths[0]:i, :-1])
        y_train.append(scaled_train_data[i, -1])
    X_train, y_train = np.array(X_train), np.array(y_train)

    scaled_test_data = scaler.transform(df_test_subset[features + [target]])

    # Testsequenzen erstellen
    X_test, y_test = [], []
    for i in range(sequence_lengths[0], len(scaled_test_data)):
        X_test.append(scaled_test_data[i-sequence_lengths[0]:i, :-1])
        y_test.append(scaled_test_data[i, -1])
    X_test, y_test = np.array(X_test), np.array(y_test)

    param_combinations = list(product(
        epochs_options, batch_sizes, dropout_rates_1, dropout_rates_2,
        regularizers_faktor, lstm_units_1, lstm_units_2
    ))

    for epochs, batch_size, dropout_rate_1, dropout_rate_2, reg_factor, lstm_units_1, lstm_units_2 in param_combinations:
        print(f"Trainiere Modell mit Epochs: {epochs}, Batch Size: {batch_size}, "
              f"regularizers_faktor: {reg_factor}, Dropout Rate 1: {dropout_rate_1}, Dropout Rate 2: {dropout_rate_2}")

        model = Sequential([
            Input(shape=(X_train.shape[1], X_train.shape[2])),
            LSTM(lstm_units_1, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(reg_factor)),
            BatchNormalization(),
            Dropout(dropout_rate_1),
            LSTM(lstm_units_2, return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(reg_factor)),
            BatchNormalization(),
            Dropout(dropout_rate_2),
            Dense(1)
        ])

        model.compile(optimizer='adam', loss='mean_squared_error')
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2,
                  callbacks=[early_stopping], verbose=0)

        # Vorhersagen für Trainings- und Testdaten durchführen und Pinball-Score berechnen
        df_train_subset['Model_1_q50'] = np.nan
        df_train_subset.loc[df_train_subset.index[-len(y_train):], 'Model_1_q50'] = model.predict(X_train).flatten()
        
        for qu in range(10, 100, 10):
            df_train_subset[f"Model_1_q{qu}"] = df_train_subset['Model_1_q50'] * (qu / 100)
        
        train_score = pinball_score(df_train_subset, 1)
        overall_pinball_score_train = np.mean(train_score)

        df_test_subset['Model_1_q50'] = np.nan
        df_test_subset.loc[df_test_subset.index[-len(y_test):], 'Model_1_q50'] = model.predict(X_test).flatten()

        for qu in range(10, 100, 10):
            df_test_subset[f"Model_1_q{qu}"] = df_test_subset['Model_1_q50'] * (qu / 100)

        test_score = pinball_score(df_test_subset, 1)
        overall_pinball_score_test = np.mean(test_score)

        # Ergebnisse in die CSV schreiben
        with open(results_file_name, 'a') as f:
            f.write(f"{epochs},{batch_size},{sequence_lengths[0]},{dropout_rate_1},{dropout_rate_2},{reg_factor},{units_1},{units_2},{data_size},{overall_pinball_score_train},{overall_pinball_score_test}\n")

        # Ausgabe der besten Kombination
        results_df = pd.read_csv(results_file_name)
        best_result = results_df.loc[results_df['Pinball_Score_Test'].idxmin()]
        print(f"Beste Kombination: Epochs: {best_result['Epochs']}, Batch Size: {best_result['Batch_Size']}, "
              f"regularizers_faktor: {best_result['Regularizers_Faktor']}, LSTM Units 1: {best_result['LSTM_Units_1']},LSTM Units 2: {best_result['LSTM_Units_2']}, "
              f"Dropout Rate 1: {best_result['Dropout_Rate_1']}, Dropout Rate 2: {best_result['Dropout_Rate_2']}, "
              f"Pinball Score Test: {best_result['Pinball_Score_Test']}")
